<a href="https://colab.research.google.com/github/TizianoBarbari/graphoo/blob/main/Graphoo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
class Graph_Element:
    '''
    This can be either a node or an edge!
    Both types of objects hold some data and have neighbors. More precisely:

    The world of an EDGE consists exactly of its neighbors (NODES) plus its data.
    The world of a  NODE consists exactly of its neighbors (EDGES) plus its data.
    '''
    __slots__ = '_data', '_neighbors'

    def __init__(self, neighbors = set(), data = None):
        self._data = data
        self._neighbors = neighbors

    def get_data(self):
        yield self._data

    def get_neighbors(self):
        yield self._neighbors

In [2]:
from collections import defaultdict

class Graph:
    __slots__ = '_nodes', '_edges', '_node_count', '_edge_count'
    
    def __init__(self, directed = False):
        '''
        Create an empty graph. By default, it is undirected.

        This is a first basic construction.
        To do:
            - directed, multigraph...
            - make it more symmetrical in edges and node
            - algorithms
            - draw
        '''

        # Graph elements are mappings "integer_index: actual_object".
        #
        # For example:
        # self._nodes == {2: Node_2, 7: Node_7, 3: Node_3, ...}
        
        self._nodes = defaultdict(Graph_Element)
        self._edges = defaultdict(Graph_Element)
        
        # Keep track of the number of graph elements
        self._node_count = 0
        self._edge_count = 0

    def node_count(self):
        '''
        Returns the number of the nodes
        '''
        return self._node_count

    def edge_count(self):
        '''
        Returns the number of the edges
        '''
        return self._edge_count
    
    def nodes(self):
        '''
        Returns the graph's node indexes (as an iterator)
        '''
        return iter(self._nodes)

    def edges(self):
        '''
        Returns the graph's edge indexes (as an iterator)
        '''
        return iter(self._edges)

    def which_nodes(self, data):
        '''
        Returns the keys corresponding to the nodes that contain the given data.
        Returns None if no such data is found in any node.
        '''
        return (node_key for node_key in self._nodes if self._nodes[node_key]._data == data)
        # result = set()
        # for node_key in self.nodes:
        #     if self._nodes[node_key]._data == data:
        #         result.add({node_key})
        # return result
        # if result else None

    def which_edges(self, data):
        '''
        Returns the key(s) corresponding to the edge(s) that contain(s) the given data.
        Returns None if no such data is found in any edge.
        '''
        result = set()
        for edge_key in self.edges:
            if edges[edge_key]._data == data:
                result.add({edge_key})
        return result
        # if result else None

    def which_elements(self, data):
        '''
        Returns the key(s) corresponding to the Edge(s) or Node(s) that contain(s) the given data.
        Returns None if no such data is found in any edge or node.
        '''
        return self.which_edges(data).union(self.which_nodes(data))

    def get_edge_between(self, node_index_1, node_index_2, data = False):
        '''
        Returns the edge index of the edge (between_this_node_index, and_this_node_index)
        Returns None if no such edge exists.
        '''
        if node_index_1 in self._nodes:
          
            # if node_index_1 == node_index_2, we look for a loop
            if node_index_1 == node_index_2:
                first_neighbors = self._nodes[node_index_1]._neighbors
                if first_neighbors:
                    for edge in first_neighbors:
                        if self._edges[edge]._neighbors == {node_index_1}:
                            if data:
                                return edge, self._edges[edge]._data
                            return edge
                return None
                
            else:
                first_neighbors = self._nodes[node_index_1]._neighbors
                second_neighbors = self.incident_edges(node_index_2)
                if second_neighbors:
                    common_neighbors = first_neighbors.intersection(second_neighbors)
                    if common_neighbors:
                        edge = common_neighbors.pop()
                        if data:
                            return edge, self._edges[edge]._data
                        return edge
                    else:
                        return None
        return None

    def degree_node(self, node_index):
        '''
        Returns the number of edges incident on the node corresponding to node_index.
        '''
        node = self._nodes.get(node_index, None)
        if node:
            return len(self._nodes[node]._neighbors)
        else:
            return 0
        # return len(self.incident_edges(node_index))

    def degree_graph(self):
        '''
        Returns the degree of the graph, i.e. the maximum degree of its nodes.
        (Should this be stored as self._degree and updated dynamically?)
        '''
        return max(self.degree_node(node_index) for node_index in self._nodes.copy())
        # return max(self.degree_node(node_index) for node_index in list(self._nodes))
        # return max(self.degree_node(node_index) for node_index in tuple(self._nodes))

    def incident_edges(self, node_index):
        '''
        Returns the set of edges incident to node_index.
        '''
        node = self._nodes.get(node_index, None)
        if node:
            return node._neighbors
        else:
            return None

    def extremal_nodes(self, edge_index):
        '''
        Returns the nodes adjecent to the given edge_index.
        '''
        edge = self._edges.get(node_index, None)
        if edge:
            return edge._neighbors
        else:
            return None

    def adjacent_nodes(self, node_index):
        '''
        Returns the nodes (node indexes) adjacent to a given node (node index).
        '''

        adjacent_edges = self.incident_edges(node_index)
        if adjacent_edges:
            result = set()
            for edge_index in adjacent_edges:
                result = result.union(self._edges.get(edge_index)._neighbors, set()).difference({node_index})
            return result
        else:
            return None

    def get_available_node_index(self):
        '''
        Returns a free node index.
        At every moment, if the node count is N, at least one of the N + 1 integers 0, 1, ..., N is free
        '''
        return set(range(1, self._node_count + 2)).difference(self._nodes).pop()

    def get_available_edge_index(self):
        '''
        Returns a free node index.
        At every moment, if the edge count is M, at least one of the M + 1 integers 0, 1, ..., M is free
        '''
        return set(range(1, self._edge_count + 2)).difference(self._edges).pop()

    def insert_nodes(self, data, at_indexes, warn_if_index_exists = True):
        '''
        A version of self.insert_node for inserting multiple nodes.
        '''
        return [self.insert_node(data = data, at_index = at_index, warn_if_index_exists = True) for (data, at_index) in zip(data, at_indexes)]

    def insert_node(self, data = None, at_index = None, warn_if_index_exists = True):
        '''
        If at_index is None, as is probable, an index is chosen "arbitrarily" by Python set pop() method, via self.get_available_node_index.
        A non-None at_index occurs, for example, when self.insert_edge is called.
        The node_index is finally returned.
        '''

        # If an index is provided and taken, (warn and) return
        if at_index and at_index in self._nodes:
            if warn_if_index_exists:
                warning = f"Node index ({at_index}) is occupied"
                node_content = self._nodes[at_index]._data
                if node_content is None:
                    warning += " by en empty node."
                else:
                    warning += f" by (a node containing) the following data: {node_content}."
                print(warning)
            # return
            return at_index

        else:
            # Otherwise, create a new node and update the count
            new_node = Graph_Element(neighbors = set(), data = data)
            self._node_count += 1

            # If an index is provided by the user and is not taken, it is 
            if at_index:
                self._nodes[at_index] = new_node
                return at_index

            # Otherwise, let Python choose an available index in range(1, len(self._nodes) + 2), via self.get_available_node_index
            else:
                new_node_index = self.get_available_node_index()
                self._nodes[new_node_index] = new_node
                return new_node_index

    def insert_edge(self, node_index_1, node_index_2, data = None,
                            update_data = True, add_missing_endpoints = True):
        '''
        Inserts a new Edge between node_index_1 and node_index_1, possibly with data 'data'.
        '''
        # Are edges between non-existing nodes allowed?
        # Are multiple edges allowed? (Multigraph)

        # Create missing nodes
        missing_1 = node_index_1 not in self._nodes
        if missing_1:
            self.insert_node(data = None, at_index = node_index_1)

        missing_2 = node_index_2 not in self._nodes
        if missing_2:
            self.insert_node(data = None, at_index = node_index_2)

        # If either node was previously missing, the edge cannot exist; create it (this also sets the two node indexes as its neighbors):
        if missing_1 or missing_2:
            return self._create_edge(node_index_1, node_index_2, data)

        else:
            # If both nodes existed, the edge may or may not exist (if it didn't, edge_key will be None)
            edge_key = self.get_edge_between(node_index_1, node_index_2)

            # If the edge exists, either overwrite/update its data or leave it as is, then return the existing edge index
            if edge_key is not None:
                print(f"An index exists ({edge_key}) for an edge between {node_index_1} and {node_index_2}.")
                if update_data:
                    self._edges[edge_key]._data = data
                    print("Edge data updated.")
                return edge_key

            else:
                # If the edge does not exist, create it (this also sets the two indexes as neighbors)
                return self._create_edge(node_index_1, node_index_2, data)

    def _create_edge(self, node_index_1, node_index_2, data):
        '''
        # Not for user's use: it can create multiple edges, because it **assumes that both nodes exists** #
        This is called inside self.insert_edge and creates the desired edge.
        '''
        new_edge = Graph_Element({node_index_1, node_index_2}, data)

        # get a random available index
        new_edge_index = self.get_available_edge_index()

        # add the new edge to the graph
        self._edges[new_edge_index] = new_edge

        # update the edge count
        self._edge_count += 1

        # set this edge as a neighbor of the two nodes
        self._nodes[node_index_1]._neighbors.add(new_edge_index)
        self._nodes[node_index_2]._neighbors.add(new_edge_index)

        # return the new edge index
        return new_edge_index

    def delete_edge(self, node_index_1, node_index_2, but_leave_nodes = True):
        '''
        Deletes the edge between node_index_1 and node_index_2, if it exists.
        '''
        edge_index = self.get_edge_between(node_index_1, node_index_1)
        if not edge_index:
            print('Edge is missing.')
            return

        else:
            self._nodes[node_index_1]._neighbors.remove(edge_index)
            self._nodes[node_index_2]._neighbors.remove(edge_index)
            del self._edges[edge_index]
            self._edge_count -= 1

    def delete_node(self, node_index, disconnect = True):
        '''
        Deletes the node corresponding to node_index.
        '''
        # The removal of a node may imply that edges are either removed (disconnect = True, by default)...
        if disconnect:
            for edge_index in self._nodes[node_index]._neighbors:
                node_index_curr = self._edges[edge_index]._neighbors.difference({node_index})
                if node_index_curr:
                    self.delete_edge(node_index, node_index_curr.pop())
            del self._nodes[node_index]
            self._node_count -= 1

        # ... or concatenated
        else:
            pass

        return None

    def list_nodes(self):
        '''
        Prints the nodes and their data.
        '''
        node_count = self._node_count
        string = f"{self._node_count}"
        string += " NODE" if node_count == 1 else " NODES"
        print("#" * (12 + int(node_count != 1)))
        print("## " + string + " ##")
        print("#" * (12 + int(node_count != 1)) + "\n")
        header = ("Index", "Neighbors", "Data")
        print(f"{header[0]:<9} {header[1]:<14} {header[2]:<20}")
        for node_index, node in self._nodes.items():
            print(f"{node_index:<10}{str(list(node._neighbors)):<15}{str(node._data):<20}")

    def list_edges(self):
        '''
        Prints the edges and their data.
        '''
        edge_count = self._edge_count
        string = f"{self._edge_count}"
        string += " EDGE" if edge_count == 1 else " EDGES"
        print("#" * (12 + int(edge_count != 1)))
        print("## " + string + " ##")
        print("#" * (12 + int(edge_count != 1)) + "\n")
        header = ("Index", "Neighbors", "Data")
        print(f"{header[0]:<9} {header[1]:<14} {header[2]:<20}")
        for edge_index, edge in self._edges.items():
            print(f"{edge_index:<10}{str(list(edge._neighbors)):<15}{str(edge._data):<20}")

    def list_elements(self):
        '''
        Prints the graph elements.
        '''
        self.list_nodes()
        print()
        self.list_edges()

    # ALIASES
    add_vertex = insert_vertex = add_node = insert_node
    which_edge = which_edges
    which_node = which_nodes
    link_count = edge_count
    vertex_count = node_count
    list_all = list_elements

In [3]:
# Methods available to the user thus far
list(filter(lambda x: not x.startswith("_"), dir(Graph)))

['add_node',
 'add_vertex',
 'adjacent_nodes',
 'degree_graph',
 'degree_node',
 'delete_edge',
 'delete_node',
 'edge_count',
 'edges',
 'extremal_nodes',
 'get_available_edge_index',
 'get_available_node_index',
 'get_edge_between',
 'incident_edges',
 'insert_edge',
 'insert_node',
 'insert_nodes',
 'insert_vertex',
 'link_count',
 'list_all',
 'list_edges',
 'list_elements',
 'list_nodes',
 'node_count',
 'nodes',
 'vertex_count',
 'which_edge',
 'which_edges',
 'which_elements',
 'which_node',
 'which_nodes']

CREATE A GRAPH

In [4]:
g = Graph()

NODES

In [5]:
# add node
g.insert_node(data = dict(weight = 3, activation = 1))

1

In [6]:
# add node at a particular node_index (probably performed by internal methods)
g.insert_node(data = dict(weight = 3, activation = 9), at_index = 6)

6

In [7]:
# use an alias as well
g.add_node(data = dict(weight = 5, activation = 4))

2

In [8]:
# add empty node
g.insert_node()

3

In [9]:
# if chosen index is occupied, no node is added
g.add_node(data = dict(weight = 5, activation = 4), at_index = 3)

Node index (3) is occupied by en empty node.


3

In [10]:
g.add_node(data = dict(weight = 5, cats = 4), at_index = 4)

4

In [11]:
# add multiple nodes at particular indexes: only free indexes are used
g.insert_nodes(data = ['A_9', "A_8", "Book 7", "Pamphlet 6", "Magazine 5"], at_indexes = (1,2,9,11,5))

Node index (1) is occupied by (a node containing) the following data: {'weight': 3, 'activation': 1}.
Node index (2) is occupied by (a node containing) the following data: {'weight': 5, 'activation': 4}.


[1, 2, 9, 11, 5]

EDGES

In [12]:
# add edge between certain nodes
g.insert_edge(2, 2, data = "Alice")

1

In [13]:
# retrieve the edge, with its data
g.get_edge_between(2, 2, data = True)

(1, 'Alice')

In [14]:
# trying to add an edge where it already existed (for now edges don't have a multiplicity)
g.insert_edge(2, 2, data = "Bob")
g.get_edge_between(2, 2, data = True)

An index exists (1) for an edge between 2 and 2.
Edge data updated.


(1, 'Bob')

In [15]:
g.insert_edge(1, 2, "1 -> 2")

2

In [16]:
# the graph is unordered
g.insert_edge(2, 1, "1 -> 2")

An index exists (2) for an edge between 2 and 1.
Edge data updated.


2

EXPLORE

In [17]:
# List all elements
g.list_elements()

#############
## 8 NODES ##
#############

Index     Neighbors      Data                
1         [2]            {'weight': 3, 'activation': 1}
6         []             {'weight': 3, 'activation': 9}
2         [1, 2]         {'weight': 5, 'activation': 4}
3         []             None                
4         []             {'weight': 5, 'cats': 4}
9         []             Book 7              
11        []             Pamphlet 6          
5         []             Magazine 5          

#############
## 2 EDGES ##
#############

Index     Neighbors      Data                
1         [2]            Bob                 
2         [1, 2]         1 -> 2              


In [18]:
# or list just the nodes
g.list_nodes()

#############
## 8 NODES ##
#############

Index     Neighbors      Data                
1         [2]            {'weight': 3, 'activation': 1}
6         []             {'weight': 3, 'activation': 9}
2         [1, 2]         {'weight': 5, 'activation': 4}
3         []             None                
4         []             {'weight': 5, 'cats': 4}
9         []             Book 7              
11        []             Pamphlet 6          
5         []             Magazine 5          


In [19]:
# or just the edges (1 neighbor means it is a loop)
g.list_edges()

#############
## 2 EDGES ##
#############

Index     Neighbors      Data                
1         [2]            Bob                 
2         [1, 2]         1 -> 2              
